In [1]:
%cd ..

/home/melek/workspace/vscode/nb_gpu_trainer


In [2]:
!pip install -e .

Obtaining file:///home/melek/workspace/vscode/nb_gpu_trainer
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl.metadata (1.8 kB)
  Using cached GitPython-3.1.43-py3-none-any.whl.metadata (13 kB)
  Using cached protobuf-5.27.2-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
  Using cached sentry_sdk-2.10.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached gitdb-4.0.11-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.1-py3-none-any.whl.metadata (4.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 4.6 MB/s eta 0:00:0000:0100:01
Using cached click-8.1.7-py3-none-any.whl (97 kB)
Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
Using cached GitPython-3.1.43-py3-none-any.wh

# Train a tokenizer

In [3]:
from tokenizer.train import Trainer
from huggingface_hub import login
from trainer import TrainerConfig

ModuleNotFoundError: No module named 'tokenizer'

In [4]:
login()

In [5]:
def load_config(config_file)->TrainerConfig:
    import yaml

    with open(config_file) as f:
        try:
            doc = yaml.safe_load(f)
        except yaml.YAMLError as exc:
            print(exc)
            exit(1)
    
    return TrainerConfig(**doc)

trainer_cfg = load_config("config/news_trainer.yml")

In [6]:
trainer_cfg.repo_id, trainer_cfg.out_dir, trainer_cfg.ds_repo_id

('habanoz/haber-90k-gpt-v1.0', 'haber-90k-gpt', 'habanoz/haber-90k-gpt-text')

## Train and push

In [1]:
from datasets import load_dataset, DatasetDict

In [2]:
ds = load_dataset("habanoz/news-tr-90k")

In [3]:
dst = ds['train']

In [4]:
df = dst.to_pandas()

In [5]:
df.value_counts(subset=['Tag']).head(20)

Tag                     
sirkethaberleri             26526
dunya                       16083
turkiye                     10478
ekonomi                      7018
gundem                       4109
yasam                        2894
politika                     2697
futbol                       2544
kultur-sanat                 2502
koronavirus                  2230
bilim-teknoloji              1708
dunyadan-spor                1437
spor                         1239
egitim                       1148
analiz                       1126
gunun-basliklari             1100
saglik                       1042
basketbol                     819
15-temmuz-darbe-girisimi      641
analiz-haber                  497
Name: count, dtype: int64

In [6]:
dst_spor4 = dst.filter(lambda x: x['Tag'] in ['futbol','spor','basketbol','dunyadan-spor'])

In [7]:
len(dst_spor4)

6039

In [8]:
dss = dst_spor4.train_test_split(0.1)

In [9]:
dss

DatasetDict({
    train: Dataset({
        features: ['Id', 'Tag', 'Labels', 'Title', 'Summary', 'Text', '__index_level_0__'],
        num_rows: 5435
    })
    test: Dataset({
        features: ['Id', 'Tag', 'Labels', 'Title', 'Summary', 'Text', '__index_level_0__'],
        num_rows: 604
    })
})

In [10]:
def to_doc(x):
    return f"""# {x['Title']}

## Özet

{x['Summary']}

## İçerik

{x['Text']}"""

In [11]:
dss = dss.map(lambda x: {'doc':to_doc(x)}, batched=False)

Map:   0%|          | 0/5435 [00:00<?, ? examples/s]

Map:   0%|          | 0/604 [00:00<?, ? examples/s]

In [12]:
dss

DatasetDict({
    train: Dataset({
        features: ['Id', 'Tag', 'Labels', 'Title', 'Summary', 'Text', '__index_level_0__', 'doc'],
        num_rows: 5435
    })
    test: Dataset({
        features: ['Id', 'Tag', 'Labels', 'Title', 'Summary', 'Text', '__index_level_0__', 'doc'],
        num_rows: 604
    })
})

In [13]:
dss = DatasetDict({
    "train":dss['train'],
    "validation":dss['test'],
})

In [14]:
dss

DatasetDict({
    train: Dataset({
        features: ['Id', 'Tag', 'Labels', 'Title', 'Summary', 'Text', '__index_level_0__', 'doc'],
        num_rows: 5435
    })
    validation: Dataset({
        features: ['Id', 'Tag', 'Labels', 'Title', 'Summary', 'Text', '__index_level_0__', 'doc'],
        num_rows: 604
    })
})

In [15]:
dss = dss.remove_columns(['Id', 'Tag', 'Labels', 'Title', 'Summary', 'Text', '__index_level_0__'])

In [16]:
dss = dss.rename_column('doc','text')

In [17]:
dss.push_to_hub("habanoz/haber-90k-gpt-text-subset-spor4")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/habanoz/haber-90k-gpt-text-subset-spor4/commit/39a5a2e240f34a55842f375e9338ad53fffd2fa0', commit_message='Upload dataset', commit_description='', oid='39a5a2e240f34a55842f375e9338ad53fffd2fa0', pr_url=None, pr_revision=None, pr_num=None)